In [3]:
import transformers
import torch 
import gc 
import json
from tokens import *

In [4]:
gc.collect()
torch.cuda.empty_cache()

In [5]:
# Load the model and tokenizer with the access token
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    use_auth_token=access_token,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=access_token
)

# Create the pipeline with the specified model and tokenizer
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

/home/t/tzelilai/Desktop/Thesis/venv/lib64/python3.9/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
2024-11-25 17:00:38.140167: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-25 17:00:38.156038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-25 17:00:38.175246: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-25 17:00:38.18108

In [26]:
def repeater(messages):

  # Apply chat template to messages
  prompt = pipeline.tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )

  # Define terminators
  terminators = [
      pipeline.tokenizer.eos_token_id,
  ]

  # Generate text
  outputs = pipeline(
      prompt,
      max_new_tokens=2024*2,
      eos_token_id=terminators,
      do_sample=True,
      temperature=1,
      top_p=0.9,
  )
  return len(prompt), outputs


In [12]:
with open('output2.json', 'r') as file: 
    articles = json.load(file)

print(f'{len(articles[len(articles) - 1])} are available.')

5 are available.


In [24]:
content = ""
for i, article in enumerate(articles[0]):
    content += "Article" + str(i+1) + ":" + article + '\n'
# Define messages
messages = [
    {"role": "system", "content": "You are an AI assistant tasked with producing NEUTRAL news article based on the provided article summaries."
                                  "Take in consideration all the provided articles and write a new article as a journalist, ensuring the content" 
                                  "is unbiased, factual, and informative."},
                                
    {"role": "user", "content": content}
]


In [30]:
content = ""
for i, article in enumerate(articles[0]):
    content += "Article" + str(i+1) + ":" + article + '\n'
# Define messages
messages = [
    {"role": "system", "content": "You are an AI assistant tasked with producing a NEUTRAL and UNBIASED news article based on the provided article summaries. "
                                  "Your goal is to analyze all the provided information and write a single cohesive news article as a professional journalist. "
                                  "The content must be factual, concise, and informative. Do not include personal opinions or assumptions. "
                                  "Ensure the tone is formal and journalistic, suitable for a wide audience. "
                                  "Provide a clear and appropriate title for the article."},
                                
    {"role": "user", "content": content}
]


In [31]:
prompt_len, outputs = repeater(messages)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [32]:
print(outputs[0]["generated_text"][prompt_len:], end='.')

Title: Unpacking the Uncertainties Surrounding the Proposed Department of Government Efficiency

In recent weeks, the appointment of tech mogul Elon Musk and Vivek Ramaswamy to lead the Department of Government Efficiency (DOGE) has sparked a mix of curiosity and concern. While the idea of harnessing the expertise of private sector innovators to optimize government operations seems appealing, the nuances of Musk's involvement have raised questions about potential conflicts of interest and the feasibility of the initiative.

Proponents of the DOGE, like Ark Invest CEO Cathie Wood, argue that Musk's companies, including SpaceX and Tesla, possess valuable insights that can inform more efficient government spending. Wood, a longtime supporter of Musk and his companies, believes his track record of overcoming challenges in business can bring a fresh perspective to government efficiency initiatives. Musk himself aims to cut $2 trillion from the federal budget, although the feasibility and im

In [29]:
with open('generated_article.json', 'w') as file: 
    json.dump([outputs[0]["generated_text"][prompt_len:]], file, indent=1)

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [31]:
del outputs

In [1]:
del model

NameError: name 'model' is not defined